In [2]:
import sklearn
import random
import pandas as pd
from maatpy.classifiers import AdaCost
from sklearn.model_selection import StratifiedKFold
from pandas import read_csv
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier

import numpy as np
from sklearn.base import is_regressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree.tree import BaseDecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import os
from sklearn.utils import (check_random_state,
                           check_X_y,
                           check_array,
                           compute_class_weight,
                           column_or_1d)
from pycm import *
import statistics
from random import randint
from sklearn.utils.validation import check_is_fitted
from scipy.stats.mstats import gmean
from sklearn.metrics import roc_auc_score  
from sklearn.metrics import matthews_corrcoef
#############################################################################
#############################################################################
#############################################################################

# READ DATASET
dataframe=read_csv('18.csv')

dim=dataframe.shape
array = dataframe.values    
Y = array[:,dim[1]-1]
X = array[:,0:dim[1]-1]

SEED=42


AUCNU=list()
AUCNU_Whole=list()
Accuracy=list()
Accuracy_Whole=list()
Precision=list()
Precision_Whole=list()
F1=list()
F1_Whole=list()
GM=list()
GM_Whole=list()
gm_temp=list()
TPR=list()
TPR_Whole=list()
TNR=list()
TNR_Whole=list()  #Specifity
tnr_temp=list()
G=list()
G_Whole=list()
g_temp=list()
Kappa=list()   #Kappa not computed for each class - It is an overall measurement
Kappa_Whole=list() 
gr_temp = list()
GR=list()
GR_Whole=list()
Recall_MinorityClass=list()
Recall_MinorityClass_Whole=list()
Precision_MinorityClass=list()
Precision_MinorityClass_Whole=list()
F_MinorityClass=list()
F_MinorityClass_Whole=list()
MCC=list()
MCC_Whole=list()
CEN=list()
CEN_Whole=list()
temp_mauc=list()
MAUC=list()
MAUC_Whole=list()
temp_mcc_ovo=list()
MCC_OVO=list()
MCC_OVO_Whole=list()
temp_mcc_ova=list()
MCC_OVA=list()
MCC_OVA_Whole=list()


minClass=10000000;
minClass_idx=0;
from collections import Counter
s=Counter(Y)
for idx, val in enumerate(s):
    if(s[val]<minClass):
        minClass=s[val];
        minClass_idx=val;

algorithm = ['Nothing']
for alg in algorithm:
    ClassSet=list(set(Y))
    counter=0
    Total_binary_classifiers=int(len(ClassSet)*(len(ClassSet)-1)/2)
    ClassLabels = np.zeros((Total_binary_classifiers, 2))
    for k in range(10):
        SEED=randint(2, 10);
        for i in range(5):
            D_train, D_test = train_test_split(dataframe, test_size=0.20, stratify=Y, random_state=i*SEED)
            dim=D_test.shape
            Whole_preds = np.zeros((dim[0], Total_binary_classifiers))    
            Final_labels=np.zeros(dim[0]) 
            counter=0
            for n in range(0,len(ClassSet)):
                for m in range(n+1,len(ClassSet)):
                    ClassLabels[counter][0]=ClassSet[n]
                    ClassLabels[counter][1]=ClassSet[m]
                
                    DataIndex_Class1=[ind for ind, x in enumerate(D_train.values[:,dim[1]-1]) if x == ClassSet[n]]
                    DataIndex_Class2=[ind for ind, x in enumerate(D_train.values[:,dim[1]-1]) if x == ClassSet[m]] 
                    X1=D_train.iloc[DataIndex_Class1 , 0:dim[1]-1].values
                    X2=D_train.iloc[DataIndex_Class2 , 0:dim[1]-1].values
                    Y1=np.ones(len(DataIndex_Class1))
                    Y2=-(np.ones(len(DataIndex_Class2)))
                    X_train=np.concatenate((X1,X2), axis=0)
                    Y_train=np.concatenate((Y1,Y2), axis=0)                
                    X_test=D_test.iloc[:, 0:dim[1]-1].values
                    Y_test=D_test.iloc[:, dim[1]-1].values 
                    
                    classifier = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=100,
                            algorithm="SAMME", learning_rate=0.3)
                    classifier.fit(X_train, Y_train)
                    y_preds = classifier.predict(X_test) 
                    
                    for numer in range(0,len(y_preds)):
                        if(y_preds[numer]==1):
                            Whole_preds[numer][counter]=ClassLabels[counter][0]
                        else:
                            Whole_preds[numer][counter]=ClassLabels[counter][1]
                
                    counter=counter+1
                    
            #Voting Mechanism        
            for q in range(0,len(y_preds)):
                temp_list=list(Whole_preds[q])
                Final_labels[q]=int(max(set(temp_list), key = temp_list.count))
            
            Final_labels=[int(tt) for tt in Final_labels]
            cm = ConfusionMatrix(actual_vector=Y_test, predict_vector=Final_labels) 
            cm.save_html(os.path.join("","AdaBoost_Run="+str(k)+"_Fold="+str(i)))   
            
            Final_labels = np.asarray(Final_labels)
            Test_ClassSet=list(set(Y_test))
            ova_test=Y_test
            ova_pred=y_preds
            for ind1_c in range(0,len(Test_ClassSet)):
                ova_index1=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind1_c]]
                ova_index2=[ind for ind, x in enumerate(Y_test) if x != Test_ClassSet[ind1_c]]
                ova_test[ova_index1]=1
                ova_test[ova_index2]=-1
                ova_pred[ova_index1]=1
                ova_pred[ova_index2]=-1
                ova_pred[ova_index2]=0
                temp_mcc_ova.append(matthews_corrcoef(ova_test, ova_pred))
                for ind2_c in range(ind1_c+1,len(Test_ClassSet)):
                    Class1_index=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind1_c]]  
                    Class2_index=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind2_c]]  
                    test1=Y_test[Class1_index]
                    test2=Y_test[Class2_index]
                    pred1=Final_labels[Class1_index]
                    pred2=Final_labels[Class2_index]
                    test_classes=np.concatenate((test1,test2), axis=0)
                    pred_classes=np.concatenate((pred1,pred2), axis=0) 
                    for qq in range(0, len(test_classes)):
                        if(test_classes[qq]!=pred_classes[qq] and test_classes[qq]==Test_ClassSet[ind1_c]):
                            pred_classes[qq]=Test_ClassSet[ind2_c]
                        elif(test_classes[qq]!=pred_classes[qq] and test_classes[qq]==Test_ClassSet[ind2_c]):
                            pred_classes[qq]=Test_ClassSet[ind1_c]                                
                    temp_mauc.append(roc_auc_score(test_classes, pred_classes))
                    temp_mcc_ovo.append(matthews_corrcoef(test_classes, pred_classes))
                
            for idx, tt in enumerate(cm.F1):            
                gm_temp.append(cm.GM[tt])
                tnr_temp.append(cm.TNR[tt])                       
                g_temp.append(cm.G[tt])
                gr_temp.append(cm.TPR[tt])

            gm_temp=[0 if v is 'None' else v for v in gm_temp]
            gr_temp=[0 if v is 'None' else v for v in gr_temp]
            g_temp=[0 if v is 'None' else v for v in g_temp]
            tnr_temp=[0 if v is 'None' else v for v in tnr_temp]
            MCC_OVA.append(statistics.mean(temp_mcc_ova))
            temp_mcc_ova.clear()
            MAUC.append(statistics.mean(temp_mauc))
            MCC_OVO.append(statistics.mean(temp_mcc_ovo))
            temp_mcc_ovo.clear()
            temp_mauc.clear()            
            GM.append(statistics.mean(gm_temp))
            GR.append(gmean(gr_temp))
            gr_temp.clear()
            gm_temp.clear()
            G.append(statistics.mean(g_temp))
            g_temp.clear()
            Kappa.append(cm.Kappa)
            AUCNU.append(cm.AUNU)
            F1.append(cm.F1_Micro)
            Precision.append(cm.PPV_Micro)
            TPR.append(cm.TPR_Micro)
            Accuracy.append(cm.Overall_ACC)
            tnr_temp=[0 if v is 'None' else v for v in tnr_temp]
            TNR.append(statistics.mean(tnr_temp))  
            tnr_temp.clear()
            Recall_MinorityClass.append(cm.TPR[minClass_idx])
            Precision_MinorityClass.append(cm.PPV[minClass_idx])
            F_MinorityClass.append(cm.F1[minClass_idx])
            MCC.append(cm.Overall_MCC)
            CEN.append(cm.Overall_CEN)
        
        MAUC_Whole.append(statistics.mean(MAUC))
        MAUC.clear
        MCC_OVO_Whole.append(statistics.mean(MCC_OVO))
        MCC_OVO.clear()
        AUCNU=[0 if v is 'None' else v for v in AUCNU]
        AUCNU_Whole.append(statistics.mean(AUCNU))
        AUCNU.clear()    
        Accuracy_Whole.append(statistics.mean(Accuracy))
        Accuracy.clear()
        Precision=[0 if v is 'None' else v for v in Precision]
        Precision_Whole.append(statistics.mean(Precision))
        Precision.clear()     
        F1_Whole.append(statistics.mean(F1))
        F1.clear()
        GM=[0 if v is 'None' else v for v in GM]
        GM_Whole.append(statistics.mean(GM))
        GM.clear()
        TPR=[0 if v is 'None' else v for v in TPR]
        TPR_Whole.append(statistics.mean(TPR))
        TPR.clear() 
        TNR=[0 if v is 'None' else v for v in TNR]
        TNR_Whole.append(statistics.mean(TNR))
        TNR.clear()
        G=[0 if v is 'None' else v for v in G]
        G_Whole.append(statistics.mean(G))
        G.clear()
        Kappa=[0 if v is 'None' else v for v in Kappa]
        Kappa_Whole.append(statistics.mean(Kappa))
        Kappa.clear()
        Recall_MinorityClass=[0 if v is 'None' else v for v in Recall_MinorityClass]
        Recall_MinorityClass_Whole.append(statistics.mean(Recall_MinorityClass))
        Precision_MinorityClass=[0 if v is 'None' else v for v in Precision_MinorityClass]
        Precision_MinorityClass_Whole.append(statistics.mean(Precision_MinorityClass))
        F_MinorityClass_Whole.append(statistics.mean(F_MinorityClass))
        Recall_MinorityClass.clear()
        Precision_MinorityClass.clear()
        F_MinorityClass.clear()
        GR_Whole.append(statistics.mean(GR))
        GR.clear()
        MCC_Whole.append(statistics.mean(MCC))
        MCC.clear()
        CEN_Whole.append(statistics.mean(CEN))
        CEN.clear()
        MCC_OVA_Whole.append(statistics.mean(MCC_OVA))
        MCC_OVA.clear()
        
        
        
with open('AdaBoost.txt', 'w') as filehandle:
                filehandle.write('AUCNU')
        filehandle.write('\n')
        for listitem in AUCNU_Whole:
            filehandle.write('%s\t' % listitem)   
        filehandle.write('\n\n')
        filehandle.write('MAUC')
        filehandle.write('\n')
        for listitem in MAUC_Whole:
            filehandle.write('%s\t' % listitem) 
        #filehandle.write('\n\n')
        #filehandle.write('ACCURACY')
        #filehandle.write('\n')
        #for listitem in Accuracy_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('Precision')
        #filehandle.write('\n')
        #for listitem in Precision_Whole:
            #filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('F_measure'),
        filehandle.write('\n')
        for listitem in F1_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('MCC_measure'),
        filehandle.write('\n')
        for listitem in MCC_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('CEN_measure'),
        filehandle.write('\n')
        for listitem in CEN_Whole:
            filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('G_mean')
        #filehandle.write('\n')
        #for listitem in GM_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('TPR')
        #filehandle.write('\n')
        #for listitem in TPR_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('TNR')
        #filehandle.write('\n')
        #for listitem in TNR_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('G_measure')
        #filehandle.write('\n')
        #for listitem in G_Whole:
            #filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('Kappa')
        filehandle.write('\n')
        for listitem in Kappa_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('GR')
        filehandle.write('\n')
        for listitem in GR_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('Recall_MinorityClass')
        filehandle.write('\n')
        for listitem in Recall_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('Precision_MinorityClass')
        filehandle.write('\n')
        for listitem in Precision_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('F_MinorityClass')
        filehandle.write('\n')
        for listitem in F_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('MCC_OVO')
        filehandle.write('\n')
        for listitem in MCC_OVO_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('\n')
        filehandle.write('MCC_OVA')
        filehandle.write('\n')
        for listitem in MCC_OVA_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')

        
        
print("%.4f" % statistics.mean(AUCNU_Whole)) #AUC_NU=
print("%.4f" % statistics.mean(MAUC_Whole)) #MAUC
#print("%.4f" % statistics.mean(Accuracy_Whole))
#print("%.4f" % statistics.mean(Precision_Whole))
print("%.4f" % statistics.mean(F1_Whole))
#print("%.4f" % statistics.mean(GM_Whole))
#print("%.4f" % statistics.mean(TPR_Whole)) #Recall
#print("%.4f" % statistics.mean(TNR_Whole)) #Specifity
#print("%.4f" % statistics.mean(G_Whole))
print("%.4f" % statistics.mean(MCC_Whole))
print("%.4f" % statistics.mean(CEN_Whole))
print("%.4f" % statistics.mean(Kappa_Whole))
print("%.4f" % statistics.mean(GR_Whole))
print("%.4f" % statistics.mean(Recall_MinorityClass_Whole))
print("%.4f" % statistics.mean(Precision_MinorityClass_Whole))
print("%.4f" % statistics.mean(F_MinorityClass_Whole))
print("%.4f" % statistics.mean(MCC_OVO_Whole))
print("%.4f" % statistics.mean(MCC_OVA_Whole))
MCC_OVA_Whole.clear()
MCC_Whole.clear()
MAUC_Whole.clear()
F1_Whole.clear
MAUC_Whole.clear()

0.9452
0.9010
0.9017
0.8914
0.1104
0.8908
0.8976
0.9505
0.9813
0.9647
0.8088
